# Gradient Checking 

In [1]:
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu

## N-Dimensional Gradient checking

In [2]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    """
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [3]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [18]:
# Helper Functions
def dictionary_to_vector(parameters):
    """
    Roll all the parameters to a vector
    """
    keys = []
    count = 0
    for key in ['W1', 'b1', 'W2', 'b2', 'W3', 'b3']:
        # flatten parameters
        new_vector = parameters[key].reshape((-1,1))
        keys = keys + [key]*new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count += 1
    print(keys)
    return theta

def gradients_to_vector(gradients):
    """
    Roll all the gradients to vector
    """
    count = 0
    for key in ['dW1', 'db1', 'dW2', 'db2', 'dW3', 'db3']:
        # flatten gradients
        new_vector = gradients[key].reshape((-1, 1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count += 1
    return theta

def vector_to_dictionary(theta):
    """
    Unroll all parameters to dictionary
    """
    parameters = {}
    parameters["W1"] = theta[:20].reshape((5,4))
    parameters["b1"] = theta[20:25].reshape((5,1))
    parameters["W2"] = theta[25:40].reshape((3,5))
    parameters["b2"] = theta[40:43].reshape((3,1))
    parameters["W3"] = theta[43:46].reshape((1,3))
    parameters["b3"] = theta[46:47].reshape((1,1))

    return parameters
    

In [19]:
# Gradient Check
def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output
    
    Arguments:
    parameters -- python dictionary containing your parameters W1, b1, W2, b2...
    grad -- output of backward_propagation_n, contains gradientes of the cost with respect to the parameters
    X -- input data, of shape (input size, number of examples)
    Y -- true label
    epsilon -- tiny shift to the input to compute approximated gradient
    
    Returns:
    difference -- difference between the approximated gradient and the backward gradient
    """
    
    # Set up variables
    parameters_values= dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # Compute Gradapprox
    for i in range(num_parameters):
        # Compute J_plus[i]
        # Copy parameters_values to thetaplus to keep the original values
        thetaplus = np.copy(parameters_values)
        # Add thetaplus[i][0] by epsilon
        thetaplus[i][0] += epsilon
        # Compute J with forward propagation
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))
        
        # Compute J_minus[i] same way as above
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] -= epsilon
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))
        
        # Compute gradapprox
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        
    # Compare gradapprox with gradient
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator
    
    if difference > epsilon:
        print('There is a mistake in a backward propagation! difference = ' + str(difference))
    else:
        print('It is perfect! difference = ' + str(difference))
        
    return difference

In [20]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

['W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'W1', 'b1', 'b1', 'b1', 'b1', 'b1', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'W2', 'b2', 'b2', 'b2', 'W3', 'W3', 'W3', 'b3']
There is a mistake in a backward propagation! difference = 0.285093156654
